In [20]:
# import random
# import operator
# import numpy as np
# import pandas as pd
# # from binarytree import Node,build
# import random
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
import operator


from deap.base import *
from deap.creator import *
from deap.gp import *
from deap.tools import *



def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = PrimitiveSet("main", 9)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
# pset.addEphemeralConstant("randconst", lambda: random.randint(-1,1))


# pset = gp.PrimitiveSet("main", 9)
# pset.addPrimitive(max, 2)
# pset.addPrimitive(operator.add, 2)
# pset.addPrimitive(operator.mul, 2)
# a=4
# pset.addTerminal(4)

pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


expr = gp.genFull(pset, min_=1, max_=5)
tree = gp.PrimitiveTree(expr)


def treeEval(individual, points):
    func = compile(individual, pset)
    return 0


# func = compile(tree, pset)
# print(tree)
# print(func(1,2,3,4,5,6,7,8,9))



creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

toolbox = base.Toolbox()

toolbox.register("expr", gp.genFULL, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)





# toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
# toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



# x=toolbox.population()
# print(x[1][1])
# print(x[9][9])

# tree = gp.PrimitiveTree(expr)

# function = gp.compile(str(x[1][1]), pset)


mul(mul(se, se), add(shape, bn))
225
